In [1]:
import openpyxl
from openpyxl.utils import get_column_letter
from openpyxl.styles import PatternFill

In [2]:
test = openpyxl.load_workbook('data/test.xlsx', data_only=True)
sheet_test = test['Лист1']

global_hot = openpyxl.load_workbook('data/Global_Hot_List.xlsx', data_only=True)
hotlist_compl = global_hot['Hotlist']



In [3]:
def meta_parser(sheet):
    cell_info = {}
    for row in sheet.iter_rows():
        for cell in row:
            cell_address = cell.coordinate
    
            cell_value = cell.value
    
            cell_data_type = type(cell_value).__name__
    
            borders = {
                "top": cell.border.top.style if cell.border.top.style else None,
                "right": cell.border.right.style if cell.border.right.style else None,
                "bottom": cell.border.bottom.style if cell.border.bottom.style else None,
                "left": cell.border.left.style if cell.border.left.style else None
            }
    
    
            column_index = cell.column
            row_index = cell.row
    
            def get_value(sheet, column_index, row_index):
                if column_index < 1 or row_index < 1:
                    return None
                return sheet[get_column_letter(column_index) + str(row_index)].value
    
            neighbor_cells = {
                "above": get_value(sheet, column_index, row_index - 1),
                "below": get_value(sheet, column_index, row_index + 1),
                "left": get_value(sheet, column_index - 1, row_index),
                "right": get_value(sheet, column_index + 1, row_index),
                "upper_left": get_value(sheet, column_index - 1, row_index - 1),
                "upper_right": get_value(sheet, column_index + 1, row_index - 1),
                "lower_left": get_value(sheet, column_index - 1, row_index + 1),
                "lower_right": get_value(sheet, column_index + 1, row_index + 1)
            }
    
            cell_info[cell_address] = {
                "value": cell_value,
                "data_type": cell_data_type,
                "borders": borders,
                "neighbors": neighbor_cells
            }
    
    for cell_address, info in cell_info.items():
        print(f"Ячейка {cell_address}:")
        print(f"Значение: {info['value']}")
        print(f"Тип данных: {info['data_type']}")
        print(f"Границы: {info['borders']}")
        print(f"Соседние ячейки: {info['neighbors']}")
        print()

In [4]:
meta_parser(sheet_test)

Ячейка A1:
Значение: A
Тип данных: str
Границы: {'top': 'thin', 'right': 'thin', 'bottom': 'thin', 'left': 'thin'}
Соседние ячейки: {'above': None, 'below': 'x', 'left': None, 'right': 'B', 'upper_left': None, 'upper_right': None, 'lower_left': None, 'lower_right': 'x'}

Ячейка B1:
Значение: B
Тип данных: str
Границы: {'top': 'thin', 'right': 'thin', 'bottom': 'thin', 'left': 'thin'}
Соседние ячейки: {'above': None, 'below': 'x', 'left': 'A', 'right': 'C', 'upper_left': None, 'upper_right': None, 'lower_left': 'x', 'lower_right': 'x'}

Ячейка C1:
Значение: C
Тип данных: str
Границы: {'top': 'thin', 'right': 'thin', 'bottom': 'thin', 'left': 'thin'}
Соседние ячейки: {'above': None, 'below': 'x', 'left': 'B', 'right': 'D', 'upper_left': None, 'upper_right': None, 'lower_left': 'x', 'lower_right': 'x'}

Ячейка D1:
Значение: D
Тип данных: str
Границы: {'top': 'thin', 'right': 'thin', 'bottom': 'thin', 'left': 'thin'}
Соседние ячейки: {'above': None, 'below': 'x', 'left': 'C', 'right': None

In [5]:
def detection_by_empty_row(sheet):
    def is_row_empty(row):
        for cell in row:
            value = cell.value
            left = cell.border.left.style if cell.border.left.style else None
            right = cell.border.right.style if cell.border.right.style else None

            if any(item is not None for item in [value, left, right]):
                return False
        return True
       

    def is_column_empty(table, column_index):
        for row in table:
            value = row[column_index].value
            left = row[column_index].border.top.style if row[column_index].border.top.style else None
            right = row[column_index].border.bottom.style if row[column_index].border.bottom.style else None
            
            if any(item is not None for item in [value, left, right]):
                    return False
        return True
    
    def split_table_by_empty_columns(table):
        subtables = []
        current_subtable = []
        columns_count = len(table[0])
        
        for end_col in range(columns_count):
            if is_column_empty(table, end_col):
                if current_subtable:  
                    subtables.append(current_subtable)
                    current_subtable = []
            else:
                for row_index, row in enumerate(table):
                    if len(current_subtable) <= row_index:
                        current_subtable.append([])
                    current_subtable[row_index].append(row[end_col].value)
        
        if current_subtable:
            subtables.append(current_subtable)
        
        return subtables
    
    tables = []
    current_table = []
    for row in sheet.iter_rows():
        if is_row_empty(row):
            if current_table:
                tables.extend(split_table_by_empty_columns(current_table))
                current_table = []
        else:
            current_table.append(row)
    if current_table:
        tables.extend(split_table_by_empty_columns(current_table))
    
    for i, table in enumerate(tables):
        print(f"Таблица {i+1}:")
        for row in table:
            print(row)

In [6]:
detection_by_empty_row(sheet_test)

Таблица 1:
['A', 'B', 'C', 'D']
['x', 'x', 'x', 'x']
['x', 'x', 'x', 'x']
['x', 'x', 'x', 'x']
['x', 'x', 'x', 'x']
['x', 'x', 'x', 'x']
Таблица 2:
['B5', 'C5', 'D5']
['x', 'x', 'x']
['x', 'x', 'x']
['x', 'x', 'x']
['x', 'x', 'x']
['x', 'x', 'x']
Таблица 3:
['A2', 'B2', 'C2', 'D2']
['x', 'x', 'x', 'x']
['x', 'x', 'x', 'x']
['x', 'x', 'x', 'x']
['x', 'x', 'x', 'x']
['x', 'x', 'x', 'x']
Таблица 4:
['B3', 'C3', 'D3']
['x', 'x', 'x']
['x', None, 'x']
['x', None, 'x']
['x', 'x', None]
['x', None, 'x']
Таблица 5:
['A4', 'B4', 'C4', 'D4', 'E4']
[None, None, None, 'x', 'x']
['x', 'x', None, 'x', 'x']
['x', 'x', None, None, 'x']
['x', None, None, None, 'x']
['x', None, 'x', None, 'x']
['x', None, None, None, 'x']
['x', None, None, None, 'x']
['x', 'x', 'x', 'x', 'x']


In [7]:
detection_by_empty_row(hotlist_compl)

Таблица 1:
['E N R O N   G L O B A L   M A R K E T S  -  H O T  L I S T']
[None]
Таблица 2:
['1Q01 DEALS COMPLETED']
['Results based on activity through March 2, 2001']
Таблица 3:
['Crude & Products', None, None, None, None]
['Deal', None, 'Value', None, 'Deal Count']
['Lubrizol', None, 8000, None, None]
['Envera', None, 989, None, None]
['Peerless', None, 500, None, None]
[None, None, None, None, None]
['SUBTOTAL', None, 9489, None, 3]
Таблица 4:
['Freight', None, None, None, None]
['Deal', None, 'Value', None, 'Deal Count']
[None, None, None, None, None]
[None, None, None, None, None]
[None, None, None, None, None]
[None, None, None, None, None]
['SUBTOTAL', None, 0, None, 0]
Таблица 5:
['Coal', None, None, None, None]
['Deal', None, 'Value', None, 'Deal Count']
[None, None, None, None, None]
[None, None, None, None, None]
['SUBTOTAL', None, 0, None, 0]
Таблица 6:
['LNG', None, None, None, None]
['Deal', None, 'Value', None, 'Deal Count']
['Hoegh Galleon', None, 801, None, None]
[Non